In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
heart_data = pd.read_csv('Fichiers_csv/heart.csv')

In [4]:
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [5]:
heart_data.shape

(1025, 14)

In [6]:
heart_data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [7]:
heart_data['target'].value_counts()

target
1    526
0    499
Name: count, dtype: int64

In [8]:
X = heart_data.drop(columns='target', axis=1)
Y = heart_data['target']

In [9]:
print(X)

      age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0      52    1   0       125   212    0        1      168      0      1.0   
1      53    1   0       140   203    1        0      155      1      3.1   
2      70    1   0       145   174    0        1      125      1      2.6   
3      61    1   0       148   203    0        1      161      0      0.0   
4      62    0   0       138   294    1        1      106      0      1.9   
...   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
1020   59    1   1       140   221    0        1      164      1      0.0   
1021   60    1   0       125   258    0        0      141      1      2.8   
1022   47    1   0       110   275    0        0      118      1      1.0   
1023   50    0   0       110   254    0        0      159      0      0.0   
1024   54    1   0       120   188    0        1      113      0      1.4   

      slope  ca  thal  
0         2   2     3  
1         0   0     3  
2  

In [10]:
X = np.asarray(X)
Y = np.asarray(Y)

### Model Selection

In [11]:
models = [LogisticRegression(max_iter=1000), SVC(kernel='linear'), KNeighborsClassifier(), RandomForestClassifier(random_state=0)]

In [58]:
def compare_models_cv():
    
    for model in models:
        cv_score      = cross_val_score(model, X, Y, cv=5)
        mean_accuracy = round((sum(cv_score)/len(cv_score))*100,2)
        
        print('Cross Validation accuracies for the ', model, '=', cv_score)
        print('Accuracy score of the ', model, '=',mean_accuracy, '%')
        print('-------------------------------------------------------------')

In [15]:
compare_models_cv()

Cross Validation accuracies for the  LogisticRegression(max_iter=1000) = [0.88292683 0.85853659 0.87804878 0.82439024 0.80487805]
Accuracy score of the  LogisticRegression(max_iter=1000) = 84.98 %
-------------------------------------------------------------
Cross Validation accuracies for the  SVC(kernel='linear') = [0.88292683 0.86829268 0.84390244 0.81463415 0.80487805]
Accuracy score of the  SVC(kernel='linear') = 84.29 %
-------------------------------------------------------------
Cross Validation accuracies for the  KNeighborsClassifier() = [0.76585366 0.74634146 0.76097561 0.71219512 0.75121951]
Accuracy score of the  KNeighborsClassifier() = 74.73 %
-------------------------------------------------------------
Cross Validation accuracies for the  RandomForestClassifier(random_state=0) = [1.         1.         0.98536585 1.         0.98536585]
Accuracy score of the  RandomForestClassifier(random_state=0) = 99.41 %
-------------------------------------------------------------


#### Comparing the models with different Hyperparameter values using GridSearchCV

In [39]:
models_list = [LogisticRegression(max_iter=10000), 
               SVC(), 
               KNeighborsClassifier(), 
               RandomForestClassifier(random_state=0)]

In [17]:
model_hyperparameters = {
    
    'log_reg_hyperparameters': {'C': [1,5,10,20]},
    

    'svc_hyperparameters': {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                            'C': [1,5,10,20]},
    
    'KNN_hyperparameters' :{'n_neighbors': [3,5,10]},
    
    'random_forest_hyperparameters': {'n_estimators': [10,20,50,100]}
    
}

In [19]:
print(model_hyperparameters.keys())

dict_keys(['log_reg_hyperparameters', 'svc_hyperparameters', 'KNN_hyperparameters', 'random_forest_hyperparameters'])


In [20]:
print(model_hyperparameters.values())

dict_values([{'C': [1, 5, 10, 20]}, {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}, {'n_neighbors': [3, 5, 10]}, {'n_estimators': [10, 20, 50, 100]}])


In [23]:
model_hyperparameters['svc_hyperparameters']

{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}

In [21]:
model_keys = list(model_hyperparameters.keys())
print(model_keys)

['log_reg_hyperparameters', 'svc_hyperparameters', 'KNN_hyperparameters', 'random_forest_hyperparameters']


In [24]:
model_hyperparameters[model_keys[1]]

{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}

#### Applying GridSearch

In [70]:
def ModelSelection(list_of_models, hyperparameters_dictionary):
    
    result = []
    i      = 0
    
    for model in list_of_models:        
        key    = model_keys[i]
        params = hyperparameters_dictionary[key]
        i     += 1
        
        print(model)
        print(params)
        print('-------------------------------------------------')

        classifier = GridSearchCV(model, params, cv=5)
        
        classifier.fit(X, Y)

        result.append({
            'model used' : model,
            'highest score' : classifier.best_score_,
            'best hyperparameters' : classifier.best_params_
        })

    result_dataframe = pd.DataFrame(result, columns = ['model used', 'highest score', 'best hyperparameters'])

    return result_dataframe

In [71]:
ModelSelection(models_list, model_hyperparameters)

LogisticRegression(max_iter=10000)
{'C': [1, 5, 10, 20]}
-------------------------------------------------
SVC()
{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}
-------------------------------------------------
KNeighborsClassifier()
{'n_neighbors': [3, 5, 10]}
-------------------------------------------------
RandomForestClassifier(random_state=0)
{'n_estimators': [10, 20, 50, 100]}
-------------------------------------------------


,model used,highest score,best hyperparameters
0,LogisticRegression(max_iter=10000),0.849756,{'C': 1}
1,SVC(),0.845854,"{'C': 5, 'kernel': 'linear'}"
2,KNeighborsClassifier(),0.890732,{'n_neighbors': 3}
3,RandomForestClassifier(random_state=0),0.997073,{'n_estimators': 20}
